###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2022  by D. Koehn, T. Meier and J. Stampa, notebook style sheet by L.A. Barba, N.C. Clementi

In [1]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, "r").read())

# Digitale Signalbearbeitung in der Geophysik 

## Kapitel 5 Stapelung von Signalen

Bei der Stapelung wird allgemein ein Modell der Form

\begin{equation}
x_i(t) = s(t - t_i) + n_i(t) \tag{5.1}
\end{equation}

angenommen. Dabei besteht die Wertereihe $x_i$ aus einem deterministischen Signal $s(t)$, dass in der Wertereihe $i$ zeitlich um $-t_i$ verschoben ist. Es wird angenommen, dass die Form des Signals in den unterschiedlichen Wertereihen gleich ist. $n_i(t)$ ist zufälliges Rauschen, für das oft angenommen wird, dass es sich um einen Gaußschen iid-Zufallsprozess mit $E[n_i(t)] = 0$ handelt. 
Bei der Stapelung werden $M$ gemessene Wertereihen zeitlich um Schätzungen $\hat{t}_i$ verschoben und gemittelt:

\begin{equation}
\mbox{stack}(t) = \frac{1}{M} \sum_{i=1}^M x_i(t+\hat{t}_i). \tag{5.2}
\end{equation}

Es wird versucht, die Zeitverschiebung rückgängig zu machen. Anschließend werden die Wertereihen gemittelt. Die Schätzungen $\hat{t}_i$ werden für eine zu testende Hypothese berechnet. Ist sie richtig bzw. kommt sie der tatsächlichen Situation sehr nahe, liegt konstruktive Interferenz vor und $\mbox{stack}(t)$ nimmt große Werte an. Geringe Werte von $\mbox{stack}(t)$ zeigen dagegen destruktive Interferenz an und die getestete Hypothese ist eher abzulehnen. Für den Fall das $\hat{t}_i = t_i$ ist, gilt  

\begin{equation}
E[\mbox{stack}(t)] = s(t) \tag{5.3}
\end{equation}

und für große $M$ nimmt $\mbox{stack}(t)$ die Form des Signals an.

Gl. (5.2) will ich an einem einfachen Beispiel veranschaulichen. Dazu importieren wir einige `Python` Bibliotheken ...

In [2]:
# Importiere Python Bibliotheken 
# ------------------------------
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
from gsv.gsv_func import *        # Bibliothek "Geophysikalische Signalverarbeitung"

... inklusive einer neuen Bibliothek "Geophysikalische Signalverarbeitung", welche diverse Funktionen enthält, die ich speziell für die Vorlesung geschrieben hab. Die Funktion `slant_stack_intro` berechnet ein stark vereinfachtes Shotgather und stapelt die Seismogramme auf ...

In [3]:
interactive_plot = interactive(slant_stack_intro, vstack=(10., 400., 10.), stackcorr=False )
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=200.0, description='vstack', max=400.0, min=10.0, step=10.0), Checkbox…

Wie man sieht enthält das Shotgather nur eine direkte SH-Welle (links). Eine simple Stapelung aller Spuren über die Offsets (rechts) macht keinen Sinn, da man nicht über das Signal der Welle summiert, sondern zeitlich verschobene Spuren. Nach Gl. (5.2) müssen wir  zunächst den Laufzeitunterschied 

\begin{equation}
\hat{t} = - \frac{\text{offset}}{vs}\notag
\end{equation}

korrigieren, wobei $\text{offset}$ den Offset der Geophone von der Quellposition und $vs$ die S-Wellengeschwindigkeit des Untergrundes bezeichnen. Wir können diese Laufzeitkorrektur über die Option **stackcorr** aktivieren und mit dem Parameter **vstack** die Stapelgeschwindigkeit variieren. Für eine Stapelgeschwindigkeit vstack = 200 m/s wird der Laufzeitunterschied perfekt ausgeglichen und die konstruktive Interferenz führt zu einer Verstärkung des Signals. Wie in Gl. (5.3) beschrieben, ergibt der Stack die Form des Signals. 

Wir können die Kombination aus Zeitverschiebung und Stack auch so interpretieren, daß wir entlang der Laufzeitkurve der direkten Welle die Wellenform aufaddieren. Ist vstack größer oder kleiner als die optimale Stapelgeschwindigkeit, kommt es zu destruktiver Interferenz und die gestapelten Amplituden werden entsprechend kleiner. Dies können wir nutzen, um ein einfaches Werkzeug zu entwickeln, um die S-Wellengeschwindigkeit des Untergrundes basierend auf der Stapelung entlang von Laufzeitkurven zu bestimmen. 

Bevor wir uns damit genauer beschäftigen, sollen im Folgenden einige Eigenschaften der Stapelung anhand einfacher Beispiele veranschaulicht werden. 

### 5.1 Beispiele

Im einfachsten Fall ist $\hat{t}_i = 0$. Der Stack ist dann:

\begin{equation}
\mbox{stack}(t) = \frac{1}{M} \sum_{i=1}^M x_i(t). \tag{5.4}
\end{equation}

Z.B. kann durch wiederholte Anregung im Fall der  Hammerschlagseismik das Signal-Rausch-Verhältnis verbessert werden. Der Erwartungswert für den Stack beträgt:

\begin{equation}
E[\mbox{stack}(t)] = s(t). \tag{5.5}
\end{equation}

Mit zunehmendem $M$ wird das Rauschen stärker unterdrückt. Das Signal-Rausch-Verhältnis nach $M$ Messungen kann nach Abschnitt 2.3 (Signal-Rausch-Verhältnis) durch 

\begin{equation}
\frac{s_{max}}{\sigma_n(M)} \tag{5.5}
\label{eq:SNR}
\end{equation}

gemessen werden. Dabei ist $s_{max}$ der maximale Betrag des Signals und $\sigma_n(M)$ ist die erwartete Abweichung des gestapelten Rauschens von dem Erwartungswert des Rauschens nach einer Mittelung über $M$ Messungen. Der Erwartungswert des Rauschens ist Null und die Standardabweichung des Rauschens ist $\sigma_n$. Da für $\sigma_n(M)$, den Fehler in der Schätzung des Erwartungswerts nach einer Mittelung über $M$ Messungen oder den Standardfehler,

\begin{equation}
\sigma_n(M) = \frac{\sigma_n} {\sqrt{M}} \tag{5.6}
\end{equation} 

gilt, wird erwartet, dass sich das Signal-Rausch-Verhältnis proportional zu $\sqrt{M}$ verbessert:

\begin{equation}
\frac{s_{max} \sqrt{M}}{\sigma_n}. \tag{5.7}
\label{eq:E_SNR}
\end{equation}

Testen wir das Konzept am Beispiel einer Sinusfunktion auf die wir normalverteilten Noise addieren und M Zeitreihen aufstapeln. 

In [4]:
def stack(sigman, M):
    
    # Define parameters
    T = 25.                  # period 1 [s]
    dt = .1                  # time sampling [s]
    L = 500.                # length of the time series [s]

    # Define sine function ...
    omega = 2. * np.pi / T   # compute circular frequency from period
    t = np.arange(0,L+dt,dt) # compute time vector
    x = np.sin(omega*t)  # compute sine wave
        
    # Add normal distributed noise to the time series
    mu = 0.
    
    # Initialize stack
    N = len(x)
    xstack = np.zeros(N)
    for i in range(M):
    
        # Add random noise to time series
        xnoise = np.random.normal(mu, sigman, N) # create random numbers
        x1 = x + xnoise # add random noise to time series 
        
        # Stack time series
        xstack = xstack + x1 
        
    # Divide xstack by number of stacks M
    xstack = xstack / M
        
    # Initialize Plots
    plt.figure(figsize=(10,5))
    
    # Plot time series
    plt.plot(t, xstack, 'b')
    plt.xlabel('Time (s)')
    plt.ylabel('f(t) (s)')
    
    # Estimate and print S/N ratio
    SN = np.sqrt(M) / sigman 
    title = r'Stacked Time Series f(t) = Sin(t) ($S/N = \frac{smax\sqrt{M}}{\sigma_n} = $' + str(SN) + ')'
    
    plt.title(title)

Wir können die Noise Amplitude über die Standardabweichung der normalverteilten Zufallszahlen $sigman$ und die Größe des Stacks mit $M$ definieren ...

In [5]:
interactive_plot = interactive(stack, sigman=(0., 10., .5), M=(1, 1000, 10) )
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=5.0, description='sigman', max=10.0, step=0.5), IntSlider(value=491, d…

Überlagerung zweier Signale mit gleicher Frequenz $\omega_0$, aber unterschiedlicher Phase.  Das ist ein einfaches Beispiel für eine Stapelung mit $t_i\not= 0$ und $n_i=0$. Die zwei Signale können durch:

\begin{align}
s_1(t) & =\cos(\omega_0 t+\varphi_1),\tag{5.9}\\
s_2(t) & =\cos(\omega_0 t+\varphi_2)\tag{5.10}
\end{align}

beschrieben werden. Ändert sich durch die Stapelung die im Signal enthaltene Frequenz? Wovon hängt die Amplitude des Ergebnisses ab?
Nach dem Additionstheorem ergibt die Überlagerung der zwei Signale:

\begin{equation}
s_1(t)+s_2(t)=2 \cos \left(\omega_0 t + \frac {\varphi_1 + \varphi_2}{2}\right)\, \cos\left(\frac {\varphi_1 - \varphi_2}{2}\right). \tag{5.11}
\end{equation}

Das bedeutet, die Frequenz $\omega_0$ bleibt nach der Stapelung erhalten. Die im Signal vorhandenen Frequenzen ändern sich durch die Stapelung nicht. Die Phasenverschiebung wird gemittelt. Die Amplitude ist von der Phasendifferenz abhängig, da der zeitunabhängige Wichtungsfaktor  von der Phasendifferenz abhängt.  

Schauen wir uns das genauer an ...

In [6]:
def cos_phase(phi1, phi2):
    
    # Define parameters
    T = 60.                  # period 1 [s]
    dt = .1                  # time sampling [s]
    L = 200.                # length of the time series [s]

    # Define sine function ...
    omega = 2. * np.pi / T        # compute circular frequency from period
    t = np.arange(0,L+dt,dt)      # compute time vector
    s1 = np.cos(omega*t + phi1)   # compute sine wave 1
    s2 = np.cos(omega*t + phi2)   # compute sine wave 2
        
    # Initialize Plots
    plt.figure(figsize=(10,5))
    
    # Plot time series
    plt.plot(t, s1, 'b',label=r'$s_1(t) = \cos(\omega_0 t+\varphi_1)$')
    plt.plot(t, s2, 'c',label=r'$s_2(t) = \cos(\omega_0 t+\varphi_2)$')
    plt.plot(t, s1+s2, 'r',label=r'$s_1 +s_2$')
    plt.legend()
    plt.xlabel('Time (s)')
    plt.ylabel('f(t) (s)')

    title = r'Sum of Time Series $s_1(t) + s_2(t)$ ($\Delta \phi$ =' + str((phi1-phi2)/np.pi) + ' $\pi$)'    
    plt.title(title)

In [7]:
interactive_plot = interactive(cos_phase, phi1=(0.,2*np.pi,np.pi/4.), phi2=(0.,2*np.pi,np.pi/4.) )
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=3.141592653589793, description='phi1', max=6.283185307179586, step=0.7…

Für eine Phasenverschiebung von $|\varphi_1-\varphi_2|\le \frac{\pi}{2}$ (das entspricht einer Phasenverschiebung um ein Viertel der Periode oder im Orstbereich um ein Viertel der Wellenlänge) gilt für den Wichtungsfaktor $\cos \left(\frac{\varphi_1-\varphi_2}{2} \right)\ge 0.7071$ und damit liegt konstruktive Interferenz vor. Für zunehmend größere Phasenunterschiede nimmt die Amplitude des resultierenden Signals ab. Das bedeutet destruktive Interferenz. Für $\varphi_1-\varphi_2= \pi $ löschen sich die Signale gegenseitig aus. Für Phasenunterschiede größer $\pi$ nimmt aufgrund der Periodizität der Signale die Amplitude wieder zu.

Als nächstes wird die Summation von zwei Signalen mit unterschiedlicher Frequenz betrachtet:

\begin{align}
s_1(t) & = \cos(\omega_1 t),\tag{5.12}\\
s_2(t) &= \cos(\omega_2 t),\tag{5.13}
\end{align}

mit $\omega_1 > \omega_2$. Mit dem Additionstheorem folgt:

\begin{equation}
s_1(t)+s_2(t)= 2 \cos \left(\frac{\omega_1+\omega_2}{2}t\right)\, \cos \left(\frac{\omega_1-\omega_2}{2}t\right).\tag{5.14}
\label{eq:stack3}
\end{equation}

Es ergibt sich ein Signal, dass eine hochfrequente Trägerfrequenz $\frac{\omega_1+\omega_2}{2}$ enthält, die der mittleren Frequenz entspricht, und dessen Amplitude niederfrequent mit $\frac{\omega_1-\omega_2}{2}$ bzw. mit der Frequenz $f_A = \frac{1}{2\pi}\frac{\omega_1-\omega_2}{2}$ moduliert ist.

In [8]:
def cos_freq(T1, T2):
    
    # Define parameters
    dt = .1                  # time sampling [s]
    L = 200.                # length of the time series [s]

    # Define sine function ...
    t = np.arange(-L,L+dt,dt)      # compute time vector
    
    omega1 = 2. * np.pi / T1      # compute circular frequency from period 1
    omega2 = 2. * np.pi / T2      # compute circular frequency from period 2
    
    s1 = np.cos(omega1*t)   # compute sine wave 1
    s2 = np.cos(omega2*t)   # compute sine wave 2
    
    # add both sine waves
    s = s1 + s2
    
    # Fourier transform
    S = np.fft.fft(s)
    
    # Normalize by length of the time series
    N = len(s)               # length of the time series [#samples]
    S = S / (N*dt)
    
    # estimate frequencies    
    freq = np.fft.fftfreq(N, d=dt)
    
    # Initialize Plots
    plt.figure(figsize=(20,10))
    
    # Plot time series
    plt.subplot(211)
    
    plt.plot(t, s1, 'b',label=r'$s_1(t) = \cos(\omega_1 t)$')
    plt.plot(t, s2, 'c',label=r'$s_2(t) = \cos(\omega_2 t)$')
    plt.plot(t, s1+s2, 'r',label=r'$s_1 +s_2$')
    plt.legend()
    plt.xlabel('Time (s)')
    plt.ylabel('f(t) (s)')
    plt.xlim(-200,200)    
    plt.title(r'Sum of Time Series $s_1(t) + s_2(t)$')
    
    # Plot Spectrum
    plt.subplot(212)
    
    plt.plot(freq, np.abs(S), 'b')
    plt.xlabel('Freq (Hz)')
    plt.ylabel('Amplitude |X(freq)|')
    plt.title(r'Amplitude spectrum of Time Series $s_1(t) + s_2(t)$')
    plt.xlim(0., .125)

In [9]:
interactive_plot = interactive(cos_freq, T1=(0.,100.,1.), T2=(0.,120.,1.) )
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=50.0, description='T1', step=1.0), FloatSlider(value=60.0, description…

Interessant ist, dass bei $\frac{\omega_1-\omega_2}{2}t = \frac{\pi}{2}  + n\pi$ mit  $n \in \mathbb{G}$ sich das Vorzeichen der Amplitudenmodulation ändert. Dadurch kommt es zu den Zeitpunkten $\frac{1}{4f_A}+\frac{n}{2f_A}$ zu einem Phasensprung im resultierenden Signal. In diesem Beispiel ($T_1$=50 s, $T_2$=60 s)ist das bei $\pm 150 s$ der Fall: aufeinanderfolgende Phasen zeigen die gleiche Polarität. Die Maxima der Einhüllenden sind $\frac{1}{2f_A}$ entfernt. Sind die Frequenzen ähnlich, ergeben sich sogenannte Schwebungen, z.B. für $T_1$=11 s, $T_2$=12 s. 

Beachte: wird eine Fourieranalyse von diesem Signal durchgeführt, treten nur die Frequenzen $\omega_1$ und $\omega_2$ auf und nicht die mittlere Trägerfrequenz bzw. die Frequenz der Amplitudenmodulation. Die Formulierung mit Hilfe der amplitudenmodulierter Trägerfrequenz ist  mathematisch äquivalent zu der Addition der beiden Signale. Da bei der Fourieranalyse eine Funktion durch Summation von Cosinus-Funktionen verschiedener Frequenz dargestellt wird, werden die sich überlagernden Frequenzen erkannt.

Im nächsten Beispiel soll ein Signal zwei Frequenzen $\omega_1$ und $\omega_2$, mit $\omega_1<\omega_2$ enthalten: $s(t) = \cos (\omega_1t) + \cos(\omega_2t)$. Betrachtet wird die Überlagerung dieses Signals mit sich selbst, wobei das zweite Signal gegenüber dem ersten um $\tau$ zeitverschoben ist: 

\begin{align}
s_1(t) &=s(t),\tag{5.15}\\
s_2(t) &=s(t+\tau).\tag{5.16}
\end{align}

In [10]:
def cos_freq_tau(T1, T2,tau):
    
    # Define parameters
    dt = .1                  # time sampling [s]
    L = 180.                 # length of the time series [s]

    # Define sine function ...
    t = np.arange(0.,L+dt,dt)      # compute time vector
    
    omega1 = 2. * np.pi / T1      # compute circular frequency from period 1
    omega2 = 2. * np.pi / T2      # compute circular frequency from period 2
    
    s1 = np.cos(omega1*t) + np.cos(omega2*t) # compute sine wave 1
    s2 = np.cos(omega1*(t+tau)) + np.cos(omega2*(t+tau))   # compute sine wave 2
    
    s = s1 + s2
    
    # Fourier transform
    S = np.fft.fft(s)
    
    # Normalize by length of the time series
    N = len(s)               # length of the time series [#samples]
    S = S / (N*dt)
    
    # estimate frequencies    
    freq = np.fft.fftfreq(N, d=dt)
    
    # Initialize Plots
    plt.figure(figsize=(20,10))
    
    # Plot time series
    plt.subplot(211)
    
    plt.plot(t, s1, 'b',label=r'$s_1(t) = \cos(\omega_1 t) + \cos(\omega_2 t)$')
    plt.plot(t, s2, 'c',label=r'$s_2(t) = \cos(\omega_1 (t+\tau)) + \cos(\omega_2 (t+\tau))$')
    plt.plot(t, s, 'r',label=r'$s_1 +s_2$')
    plt.legend()
    plt.xlabel('Time (s)')
    plt.ylabel('f(t) (s)')
    plt.xlim(0.,180)    
    plt.title(r'Sum of Time Series $s_1(t) + s_2(t)$')
    
    # Plot Spectrum
    plt.subplot(212)
    
    plt.plot(freq, np.abs(S), 'b')
    plt.xlabel('Freq (Hz)')
    plt.ylabel('Amplitude |X(freq)|')
    plt.title(r'Amplitude spectrum of Time Series $s_1(t) + s_2(t)$')
    plt.xlim(0., .125)

In [11]:
interactive_plot = interactive(cos_freq_tau, T1=(0.,40.,1.), T2=(0.,160.,1.), tau=(0.,16.,1.))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=20.0, description='T1', max=40.0, step=1.0), FloatSlider(value=80.0, d…

Wie wirkt sich die Zeitverschiebung auf die beiden Frequenzen aus? Mit dem Additionstheorem folgt:

\begin{equation}
s_1(t)+s_2(t)= 2 \cos \left(\omega_1 t +\omega_1\frac{\tau}{2}\right)\, \cos \left(\omega_1\frac{\tau}{2}\right)+2 \cos \left(\omega_2 t +\omega_2\frac{\tau}{2}\right)\, \cos\left(\omega_2\frac{\tau}{2}\right).\tag{5.17}
\end{equation}

- Die Frequenzen $\omega_1$ und $\omega_2$ bleiben erwartungsgemäß erhalten, da die Addition zweier Signale eine lineare Operation ist.

- Es kommt bei beiden Frequenzen zu einer Phasenverschiebung, die einer Zeitverschiebung um $\tau/2$ entspricht. 

- $\cos \left(\omega_1\frac{\tau}{2} \right)=a_1$ und $ \cos\left(\omega_2\frac{\tau}{2}\right)=a_2$ sind zeitunabhängige Wichtungsfaktoren, die für beide Frequenzen aber unterschiedlich sind. 

- Wegen $\omega_1<\omega_2$ ist $a_1>a_2$, d.h. die Stapelung ist ein Tiefpass. Die Bedingung für konstruktive Interferenz ist also für tiefe Frequenzen eher erfüllt als für hohe Frequenzen.

In [12]:
T1 = 20.
w1 = 2. * np.pi / T1

T2 = 80.
w2 = 2. * np.pi / T2

print('w1 = ', w1, '> w2 = ', w2)

w1 =  0.3141592653589793 > w2 =  0.07853981633974483


In [13]:
a1 = np.cos(w1 * 4)
a2 = np.cos(w2 * 4)

print('a1 = ', a1, '< a2 = ', a2)

a1 =  0.30901699437494745 < a2 =  0.9510565162951535


## Zusammenfassung: